# Timing Tests of `fastnumbers` Functions Compared to Equivalent Solutions

In order for you to see the benefit of `fastnumbers`, some timings are collected below for comparison to equivalent python implementations. The numbers may change depending on the machine you are on or the Python version you are using.

Feel free to download this Jupyter Notebook and run the tests yourself to see how `fastnumbers` performs on your machine (it takes about 1-2 minutes total).

**This notebook contains timing results for Python 2.7.**

#### Some notes about the data

 - Each test is the time it takes for the function to run 100,000 times on a given input.
 - Each test is repeated either 5 or 100 times, and the mean ± standard deviation is reported.
 - The fastest time is shown in **bold**
 - The timing results for the pure-Python functions include about 10-15 ms of "function call overhead"; the `fastnumbers` functions do not suffer from as much overhead because they are C-extensions.
 - Python version-dependent behaviors:
   - **Python 2.7** has a particularly slow `int` function, so the `fastnumbers` speedup is much larger on Python 2.7 than Python 3.x
   - **Python >=3.6** is slightly slower in general than previous versions because underscores are now allowed in floats and integers which makes parsing take a bit longer due to the extra logic.
   
#### Notes about the `Timing` class below

The timing runner class is implemented below, and this is used in all the tests to perform the actual timing tests in the sections below. In general you can skip this implementation, but of note is the `THINGS_TO_TIME` tuple, which contains the values that are passed to the functions to type the various input types.

In [1]:
from __future__ import print_function, division
import re
import math
import timeit
from IPython.display import Markdown, display, clear_output

class Timer(object):
    """Class to time functions and make pretty tables of the output."""
    
    # This is a list of all the things we will time with an associated label.
    THINGS_TO_TIME = (
        ('not_a_number', u'Non-number String'),
        ('-41053', u'Int String'),
        ('35892482945872302493947939485729', u'Large Int String'),
        ('-41053.543034e34', u'Float String'),
        ('-41053.543028758302e256', u'Large Float String'),
        (-41053, u'Int'),
        (-41053.543028758302e100, u'Float'),
    )

    # Formatting strings.
    FUNCTION_CALL_FMT = '{}({!r})'
    
    def __init__(self, title):
        display(Markdown('### ' + title))
        self.functions = []
    
    def add_function(self, func, label, setup='pass'):
        """Add a function to be timed and compared."""
        self.functions.append((func, setup, label))

    def time_functions(self, repeat=5):
        """Time all the given functions against all input then display results."""

        # Collect the function labels to make the header of this table.
        # Show that the units are seconds for each.
        function_labels = [label + u' (ms)' for _, _, label in self.functions]
        
        # Construct the table strings, formatted in Markdown.
        # Store each line as a string element in a list.
        # This portion here is the table header only for now.
        table = Table()
        table.add_header(u'Input type', *function_labels)
        
        # For each value, time each function and collect the results.
        for value, value_label in self.THINGS_TO_TIME:
            row = []
            for func, setup, _ in self.functions:
                call = self.FUNCTION_CALL_FMT.format(func, value)
                try:
                    row.append(self._timeit(call, setup, repeat))
                except (ValueError, TypeError):
                    # We might send in some invalid input accidentally.
                    # Ignore those inputs.
                    break

            # Only add this row if the for loop quit without break.
            else:
                # Convert to milliseconds
                row = [(mean * 1000, stddev * 1000) for mean, stddev in row]
                # Make the lowest value bold.
                min_indx = min(enumerate(row), key=lambda x: x[1])[0]
                row = [u'{:.3f} ± {:.3f}'.format(*x) for x in row]
                row[min_indx] = self.bold(row[min_indx])
                table.add_row(value_label, *row)

        # Show the results in a table.
        display(Markdown(unicode(table)))

    @staticmethod
    def mean(x):
        return math.fsum(x) / len(x)

    @staticmethod
    def stddev(x):
        mean = Timer.mean(x)
        sum_of_squares = math.fsum((v - mean)**2 for v in x)
        return math.sqrt(sum_of_squares / (len(x) - 1))

    @staticmethod
    def bold(x):
        return u"**{}**".format(x)
    
    def _timeit(self, call, setup, repeat=5):
        """Perform the actual timing and return a formatted string of the runtime"""
        result = timeit.repeat(call, setup, number=100000, repeat=repeat)
        return self.mean(result), self.stddev(result)

class Table(list):
    """List of strings that can be made into a Markdown table."""
    def add_row(self, *elements):
        self.append('|'.join(elements))
    def add_header(self, *elements):
        self.add_row(*elements)
        seperators = ['---'] * len(elements)
        seperators = [sep + (':' if i != 0 else '') for i, sep in enumerate(seperators)]
        self.add_row(*seperators)
    def __str__(self):
        return '\n'.join(self)
    def __unicode__(self):
        return u'\n'.join(self)

## Built-in Functions Drop-in Replacement Timing Results
The following timing tests compare the performance of Python's builtin `int` and `float` functions against the implementations from `fastnumbers` for various input types.

In [2]:
timer = Timer('Timing comparison of `int` functions')
timer.add_function('int', 'builtin')
timer.add_function('int', 'fastnumbers', 'from fastnumbers import int')
timer.time_functions(repeat=100)

### Timing comparison of `int` functions

Input type|builtin (ms)|fastnumbers (ms)
---|---:|---:
Int String|101.173 ± 13.378|**21.711 ± 2.270**
Large Int String|123.786 ± 31.828|**89.459 ± 12.439**
Int|25.424 ± 6.610|**17.122 ± 4.189**
Float|40.053 ± 9.408|**34.244 ± 4.082**

In [3]:
timer = Timer('Timing comparison of `float` functions')
timer.add_function('float', 'builtin')
timer.add_function('float', 'fastnumbers', 'from fastnumbers import float')
timer.time_functions(repeat=100)

### Timing comparison of `float` functions

Input type|builtin (ms)|fastnumbers (ms)
---|---:|---:
Int String|36.893 ± 8.293|**29.728 ± 4.165**
Large Int String|**73.612 ± 10.989**|75.805 ± 7.760
Float String|71.280 ± 16.678|**29.698 ± 4.560**
Large Float String|126.384 ± 34.909|**119.720 ± 16.692**
Int|18.345 ± 2.759|**15.505 ± 2.342**
Float|20.130 ± 4.839|**14.360 ± 2.482**

## Error-Handling Conversion Functions Timing Results
The following timing tests compare the performance of the `fastnumbers` functions that convert input to numeric types while doing error handling with common equivalent pure-Python implementations.

In [4]:
def int_re(x, int_match=re.compile(r'[-+]?\d+$').match):
    """Function to simulate fast_int but with regular expressions."""
    try:
        if int_match(x):
            return int(x)
        else:
            return x
    except TypeError:
        return int(x)

def int_try(x):
    """Function to simulate fast_int but with try/except."""
    try:
        return int(x)
    except ValueError:
        return x

timer = Timer('Timing comparison of `int` functions with error handling')
timer.add_function('int_try', 'try/except', 'from __main__ import int_try')
timer.add_function('int_re', 'regex', 'from __main__ import int_re')
timer.add_function('fast_int', 'fastnumbers', 'from fastnumbers import fast_int')
timer.time_functions()

### Timing comparison of `int` functions with error handling

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|361.431 ± 13.968|57.017 ± 3.563|**16.235 ± 1.515**
Int String|112.147 ± 19.162|170.874 ± 20.856|**19.163 ± 4.272**
Large Int String|112.679 ± 5.775|236.964 ± 14.054|**64.906 ± 5.125**
Float String|375.727 ± 22.185|88.013 ± 2.245|**20.188 ± 4.086**
Large Float String|410.145 ± 14.052|80.047 ± 7.543|**17.400 ± 2.364**
Int|27.391 ± 2.699|215.707 ± 37.494|**13.473 ± 0.019**
Float|42.734 ± 2.214|202.926 ± 12.379|**28.144 ± 4.172**

In [5]:
def float_re(x, float_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate fast_float but with regular expressions."""
    try:
        if float_match(x):
            return float(x)
        else:
            return x
    except TypeError:
        return float(x)

def float_try(x):
    """Function to simulate fast_float but with try/except."""
    try:
        return float(x)
    except ValueError:
        return x

timer = Timer('Timing comparison of `float` functions with error handling')
timer.add_function('float_try', 'try/except', 'from __main__ import float_try')
timer.add_function('float_re', 'regex', 'from __main__ import float_re')
timer.add_function('fast_float', 'fastnumbers', 'from fastnumbers import fast_float')
timer.time_functions()

### Timing comparison of `float` functions with error handling

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|234.682 ± 15.966|66.745 ± 1.314|**16.723 ± 0.307**
Int String|34.141 ± 0.352|119.113 ± 5.294|**23.219 ± 4.303**
Large Int String|61.664 ± 2.962|234.115 ± 40.272|**57.132 ± 7.420**
Float String|66.602 ± 14.432|189.457 ± 7.506|**23.185 ± 1.790**
Large Float String|120.675 ± 34.570|228.632 ± 6.617|**97.355 ± 6.959**
Int|27.653 ± 2.543|214.730 ± 28.851|**17.761 ± 2.972**
Float|27.104 ± 1.136|191.860 ± 16.932|**18.093 ± 2.613**

In [6]:
def real_re(x,
            int_match=re.compile(r'[-+]?\d+$').match,
            real_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate fast_real but with regular expressions."""
    try:
        if int_match(x):
            return int(x)
        elif real_match(x):
            return float(x)
        else:
            return x
    except TypeError:
        if type(x) in (float, int):
            return x
        else:
            raise TypeError

def real_try(x):
    """Function to simulate fast_real but with try/except."""
    try:
        a = float(x)
    except ValueError:
        return x
    else:
        b = int(a)
        return b if a == b else b

timer = Timer('Timing comparison of `float` (but coerce to `int` if possible) functions with error handling')
timer.add_function('real_try', 'try/except', 'from __main__ import real_try')
timer.add_function('real_re', 'regex', 'from __main__ import real_re')
timer.add_function('fast_real', 'fastnumbers', 'from fastnumbers import fast_real')
timer.time_functions()

### Timing comparison of `float` (but coerce to `int` if possible) functions with error handling

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|239.736 ± 26.085|121.046 ± 5.565|**20.879 ± 3.757**
Int String|69.326 ± 2.697|184.650 ± 16.253|**18.394 ± 0.072**
Large Int String|132.072 ± 13.528|232.658 ± 23.376|**70.125 ± 4.896**
Float String|151.677 ± 20.019|279.944 ± 14.248|**37.661 ± 3.033**
Large Float String|239.977 ± 3.168|296.607 ± 15.818|**118.243 ± 6.281**
Int|57.851 ± 5.062|212.900 ± 22.424|**14.739 ± 2.225**
Float|85.055 ± 6.651|185.491 ± 6.697|**42.469 ± 5.220**

In [7]:
def forceint_re(x,
                int_match=re.compile(r'[-+]\d+$').match,
                float_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate fast_forceint but with regular expressions."""
    try:
        if int_match(x):
            return int(x)
        elif float_match(x):
            return int(float(x))
        else:
            return x
    except TypeError:
        return int(x)

def forceint_try(x):
    """Function to simulate fast_forceint but with try/except."""
    try:
        return int(x)
    except ValueError:
        try:
            return int(float(x))
        except ValueError:
            return x

timer = Timer('Timing comparison of forced `int` functions with error handling')
timer.add_function('forceint_try', 'try/except', 'from __main__ import forceint_try')
timer.add_function('forceint_re', 'regex', 'from __main__ import forceint_re')
timer.add_function('fast_forceint', 'fastnumbers', 'from fastnumbers import fast_forceint')
timer.time_functions()

### Timing comparison of forced `int` functions with error handling

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|612.884 ± 11.707|114.501 ± 2.865|**20.672 ± 3.480**
Int String|110.811 ± 3.648|170.250 ± 9.307|**20.236 ± 4.279**
Large Int String|114.232 ± 6.011|275.033 ± 4.928|**68.848 ± 5.699**
Float String|545.209 ± 22.123|292.875 ± 12.875|**33.671 ± 2.678**
Large Float String|604.830 ± 37.628|350.263 ± 8.960|**123.353 ± 9.986**
Int|29.965 ± 4.202|190.830 ± 21.201|**13.246 ± 0.177**
Float|51.259 ± 12.088|204.337 ± 10.433|**23.503 ± 0.198**

## Checking Functions Timing Results
The following timing tests compare the performance of the `fastnumbers` functions that check if an input *could* be converted to numeric type with common equivalent pure-Python implementations.

In [8]:
def isint_re(x, int_match=re.compile(r'[-+]?\d+$').match):
    """Function to simulate isint but with regular expressions."""
    t = type(x)
    return t == int if t in (float, int) else bool(int_match(x))

def isint_try(x):
    """Function to simulate isint but with try/except."""
    try:
        int(x)
    except ValueError:
        return False
    else:
        return type(x) != float

timer = Timer('Timing comparison to check if value can be converted to `int`')
timer.add_function('isint_try', 'try/except', 'from __main__ import isint_try')
timer.add_function('isint_re', 'regex', 'from __main__ import isint_re')
timer.add_function('isint', 'fastnumbers', 'from fastnumbers import isint')
timer.time_functions()

### Timing comparison to check if value can be converted to `int`

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|380.387 ± 34.912|94.054 ± 3.020|**19.910 ± 4.331**
Int String|116.676 ± 10.119|126.385 ± 15.163|**17.604 ± 3.668**
Large Int String|135.369 ± 8.504|165.487 ± 10.223|**20.516 ± 3.785**
Float String|387.309 ± 27.393|118.966 ± 4.963|**17.125 ± 2.449**
Large Float String|376.728 ± 25.655|140.462 ± 11.162|**18.385 ± 2.219**
Int|59.199 ± 8.584|65.760 ± 4.455|**20.777 ± 2.508**
Float|93.483 ± 12.204|43.709 ± 0.843|**15.157 ± 2.182**

In [9]:
def isfloat_re(x, float_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate isfloat but with regular expressions."""
    t = type(x)
    return t == float if t in (float, int) else bool(float_match(x))

def isfloat_try(x):
    """Function to simulate isfloat but with try/except."""
    try:
        float(x)
    except ValueError:
        return False
    else:
        return type(x) != int

timer = Timer('Timing comparison to check if value can be converted to `float`')
timer.add_function('isfloat_try', 'try/except', 'from __main__ import isfloat_try')
timer.add_function('isfloat_re', 'regex', 'from __main__ import isfloat_re')
timer.add_function('isfloat', 'fastnumbers', 'from fastnumbers import isfloat')
timer.time_functions()

### Timing comparison to check if value can be converted to `float`

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|262.301 ± 25.634|125.525 ± 8.284|**21.082 ± 2.755**
Int String|62.725 ± 5.847|160.803 ± 6.392|**20.999 ± 1.722**
Large Int String|94.814 ± 8.252|178.786 ± 9.127|**21.557 ± 4.289**
Float String|83.937 ± 2.416|163.689 ± 6.476|**20.665 ± 3.825**
Large Float String|123.722 ± 6.166|165.443 ± 2.684|**18.597 ± 0.784**
Int|47.624 ± 4.602|40.044 ± 2.657|**13.363 ± 0.100**
Float|48.643 ± 3.788|41.654 ± 8.478|**13.820 ± 2.117**

In [10]:
def isreal_re(x, real_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate isreal but with regular expressions."""
    return type(x) in (float, int) or bool(real_match(x))

def isreal_try(x):
    """Function to simulate isreal but with try/except."""
    try:
        float(x)
    except ValueError:
        return False
    else:
        return True

timer = Timer('Timing comparison to check if value can be converted to `float` or `int`')
timer.add_function('isreal_try', 'try/except', 'from __main__ import isreal_try')
timer.add_function('isreal_re', 'regex', 'from __main__ import isreal_re')
timer.add_function('isreal', 'fastnumbers', 'from fastnumbers import isreal')
timer.time_functions()

### Timing comparison to check if value can be converted to `float` or `int`

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|238.638 ± 12.199|124.403 ± 5.983|**19.753 ± 2.578**
Int String|42.436 ± 4.072|154.051 ± 11.776|**19.090 ± 2.782**
Large Int String|61.730 ± 2.345|166.411 ± 5.562|**19.704 ± 0.165**
Float String|65.054 ± 2.892|161.396 ± 4.098|**18.965 ± 2.301**
Large Float String|111.087 ± 16.251|169.679 ± 5.046|**19.422 ± 2.438**
Int|33.288 ± 3.486|34.892 ± 2.695|**14.888 ± 2.439**
Float|31.091 ± 1.802|32.155 ± 2.482|**12.621 ± 0.189**

In [11]:
def isintlike_re(x,
                 int_match=re.compile(r'[-+]?\d+$').match,
                 float_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate isintlike but with regular expressions."""
    try:
        if int_match(x):
            return True
        elif float_match(x):
            return float(x).is_integer()
        else:
            return False
    except TypeError:
        return int(x) == x

def isintlike_try(x):
    """Function to simulate isintlike but with try/except."""
    try:
        a = int(x)
    except ValueError:
        try:
            a = float(x)
        except ValueError:
            return False
        else:
            return a.is_integer()
    else:
        return a == float(x)

timer = Timer('Timing comparison to check if value can be coerced losslessly to `int`')
timer.add_function('isintlike_try', 'try/except', 'from __main__ import isintlike_try')
timer.add_function('isintlike_re', 'regex', 'from __main__ import isintlike_re')
timer.add_function('isintlike', 'fastnumbers', 'from fastnumbers import isintlike')
timer.time_functions()

### Timing comparison to check if value can be coerced losslessly to `int`

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|576.280 ± 53.449|108.002 ± 10.835|**17.619 ± 3.504**
Int String|133.047 ± 7.734|81.662 ± 9.826|**19.764 ± 2.615**
Large Int String|240.449 ± 15.356|106.008 ± 2.920|**22.428 ± 3.456**
Float String|532.435 ± 38.686|282.126 ± 26.171|**17.174 ± 0.052**
Large Float String|544.959 ± 19.542|321.752 ± 19.000|**19.837 ± 3.997**
Int|53.256 ± 5.187|185.891 ± 4.799|**12.830 ± 0.615**
Float|82.127 ± 3.947|237.837 ± 7.590|**29.697 ± 1.967**